# Debug Experiments Configuration

This notebook sets up debug experiments across multiple datasets with their specific model architectures:
- CIFAR-100: WideResNet (wrn-28-10)
- GTSRB: Custom CNN
- ImageNette: ResNet50

In [ ]:
import torch
from pathlib import Path
from config.experiment_config import create_config
from utils.logging import setup_logging, get_logger

# Initialize logging
setup_logging()
logger = get_logger(__name__)

## Checkpoint Configuration

Find checkpoints for each model architecture:

In [ ]:
def find_checkpoint(model_type):
    """Find the best or latest checkpoint for a model."""
    checkpoint_dir = Path('~/Notebooks/classify/checkpoints') / model_type
    checkpoint_dir = checkpoint_dir.expanduser()
    
    if not checkpoint_dir.exists():
        logger.warning(f"Checkpoint directory {checkpoint_dir} does not exist")
        return None
        
    # First try to find best checkpoint
    best_checkpoint = checkpoint_dir / f'{model_type}_best.pt'
    if best_checkpoint.exists():
        logger.info(f"Found best checkpoint: {best_checkpoint}")
        return best_checkpoint
        
    # Otherwise get latest checkpoint
    latest_checkpoint = checkpoint_dir / f'{model_type}_latest.pt'
    if latest_checkpoint.exists():
        logger.info(f"Found latest checkpoint: {latest_checkpoint}")
        return latest_checkpoint
        
    logger.warning(f"No checkpoints found in {checkpoint_dir}")
    return None

# Find checkpoints for each model type
checkpoints = {
    'cifar100': find_checkpoint('wideresnet'),
    'gtsrb': find_checkpoint('custom-cnn'),
    'imagenette': find_checkpoint('resnet50')
}

## Hardware Configuration

Set up hardware-specific configurations:

In [ ]:
# Determine hardware configuration
if torch.cuda.is_available():
    device_info = f"CUDA (GPU: {torch.cuda.get_device_name(0)})"
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    gpu_ids = [0]
elif torch.backends.mps.is_available():
    device_info = "MPS (Apple Silicon)"
    gpu_ids = []
else:
    device_info = "CPU"
    gpu_ids = []

print(f"Running on: {device_info}")

# Set hardware-dependent parameters
hardware_config = {
    'execution': {
        'max_workers': 1,
        'gpu_ids': gpu_ids
    },
    'training': {
        'num_workers': 4 if not torch.backends.mps.is_available() else 0,
        'pin_memory': True
    }
}

## Dataset Configurations

Create configurations for each dataset with their specific model architectures:

In [ ]:
# CIFAR-100 with WideResNet
cifar_config = create_config(
    'cifar100',
    **hardware_config,
    training={'epochs': 200, 'batch_size': 128},
    data={'subset_size': 50000},
    checkpoint={
        'save_dir': str(checkpoints['cifar100'].parent) if checkpoints['cifar100'] else 'checkpoints',
        'resume': True if checkpoints['cifar100'] else False
    },
    poison={
        'poison_type': 'ga',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'ga_steps': 50,
        'ga_iterations': 100,
        'ga_lr': 0.1
    }
)

# GTSRB with Custom CNN
gtsrb_config = create_config(
    'gtsrb',
    **hardware_config,
    training={'epochs': 10, 'batch_size': 128},
    data={'subset_size': 39209},
    checkpoint={
        'save_dir': str(checkpoints['gtsrb'].parent) if checkpoints['gtsrb'] else 'checkpoints',
        'resume': True if checkpoints['gtsrb'] else False
    },
    poison={
        'poison_type': 'pgd',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'pgd_eps': 0.3,
        'pgd_alpha': 0.01,
        'pgd_steps': 40
    }
)

# ImageNette with ResNet50
imagenette_config = create_config(
    'imagenette',
    **hardware_config,
    training={'epochs': 10, 'batch_size': 64},
    data={'subset_size': 9469},
    checkpoint={
        'save_dir': str(checkpoints['imagenette'].parent) if checkpoints['imagenette'] else 'checkpoints',
        'resume': True if checkpoints['imagenette'] else False
    },
    poison={
        'poison_type': 'pgd',
        'poison_ratio': 0.1,
        'batch_size': 32,
        'pgd_eps': 0.3,
        'pgd_alpha': 0.01,
        'pgd_steps': 40
    }
)

print("Model architectures and dataset sizes:")
print(f"CIFAR-100: {cifar_config.model} (size: {cifar_config.data.subset_size})")
print(f"GTSRB: {gtsrb_config.model} (size: {gtsrb_config.data.subset_size})")
print(f"ImageNette: {imagenette_config.model} (size: {imagenette_config.data.subset_size})")

## Create Experiment Groups

Set up experiment groups for comparing different attacks:

In [ ]:
# Create experiment groups
experiment_groups = {
    'cifar100': {
        'description': 'CIFAR-100 experiments with WideResNet',
        'experiments': [cifar_config]
    },
    'gtsrb': {
        'description': 'GTSRB experiments with Custom CNN',
        'experiments': [gtsrb_config]
    },
    'imagenette': {
        'description': 'ImageNette experiments with ResNet50',
        'experiments': [imagenette_config]
    }
}

print("\nExperiment Groups:")
for name, group in experiment_groups.items():
    print(f"\n{name}: {group['description']}")
    print(f"Number of experiments: {len(group['experiments'])}")
    print(f"Dataset size: {group['experiments'][0].data.subset_size}")